## PMC Instrumentation
* requirements:
    * mlib_devel : m2019a branch
    * casperfpga : py38 branch, snapadc.py has to be changed

### Step1: Connect to the SNAP board 

In [ ]:
import casperfpga
import logging
import time
pmc_ip = '192.168.2.102'
port   = 69
logger=logging.getLogger('pmc')
logging.basicConfig(filename='pmc.log',level=logging.DEBUG)
pmc=casperfpga.CasperFpga(pmc_ip, port, logger=logger)

### Step2: Upload fpg file

In [ ]:
fpg = 'fpg/dsa10_2022-11-02_1656.fpg'
pmc.upload_to_ram_and_program(fpg)
# We should get system info in "upload_to_ran_and_program", but it seems there are some issues in the casperfpga
pmc.get_system_information(fpg,initialise_objects=False)
pmc.listdev()

### Step3: Init adc and clk

In [ ]:
adc=pmc.adcs['snap_adc']
adc.selectADC()
adc.init(sample_rate=500,numChannel=2)
# set adc scales
# To-do: scale value for each channel is a 3-bits value
pmc.registers['scaling'].write_int(0)
# set delays between adc module and pfb_fir module
pmc.registers['del1'].write_int(5)
pmc.registers['del2'].write_int(5)
pmc.registers['del3'].write_int(5)
pmc.registers['del4'].write_int(5)
pmc.registers['del5'].write_int(5)
pmc.registers['del6'].write_int(5)
pmc.registers['del7'].write_int(5)
pmc.registers['del8'].write_int(5)

### Step4: Configure basic registers

In [ ]:
# set snap_index
pmc.registers['snap_index'].write_int(0)
# set fft shift
pmc.registers['fft_shift'].write_int(65535)
# set sel, which is used for selecting 16bits from 64-bit spectra data
# sel: '0' selects 15-0 bit
#      '1' selects 31:16 bit
#      '2' selects 47-32 bit
#      '3' selects 63-48 bit
pmc.registers['sel1'].write_int(1)
# set coeff, which is the coefficient for the 64-bit spectra data
pmc.registers['coeff1'].write_int(7)

### Step5: Configure 10GbE port

In [ ]:
gbe0=pmc.gbes['eth_gbe0']
gbe1=pmc.gbes['eth1_gbe1']

# configure gbe0
# src
gbe0_src_mac  = "00:08:0b:c4:17:00"
gbe0_src_ip   = "192.168.2.101"
gbe0_src_port = 4001
gbe0.configure_core(gbe0_src_mac, gbe0_src_ip, gbe0_src_port)
# dst
gbe0_dst_mac  = 0xf452141624a0
# write register requires a int vaule, but set_single_arp_entry requires a string
gbe0_dst_ip   = 192*(2**24) + 168*(2**16) + 2*(2**8) + 1
gbe0_dst_ip_str='192.168.2.1'
gbe0.set_single_arp_entry(gbe0_dst_ip_str,gbe0_dst_mac)
gbe0.fabric_disable()

# configure gbe1
# src
gbe1_src_mac  = "00:08:0b:c4:17:01"
gbe1_src_ip   = "192.168.2.102"
gbe1_src_port = 4000
gbe1.configure_core(gbe1_src_mac, gbe1_src_ip, gbe1_src_port)
# dst
gbe1_dst_mac  = 0xf452141624a0
# write register requires a int vaule, but set_single_arp_entry requires a string
gbe1_dst_ip   = 192*(2**24) + 168*(2**16) + 2*(2**8) + 1
gbe1_dst_ip_str='192.168.2.1'
gbe1.set_single_arp_entry(gbe1_dst_ip_str,gbe1_dst_mac)
gbe1.fabric_enable()
pmc.registers['eth1_ctrl'].write_int(1+ 2 + (1<<18))
time.sleep(0.1)
pmc.registers['eth1_ctrl'].write_int(0 +2 + (0<<18))

### Step6 : Configre integration time and then rst the system

In [ ]:
# set acc len
pmc.registers['acc_len'].write_int(127)
# full rst
pmc.registers['force_sync'].write_int(2) 
time.sleep(0.1)
# sync
pmc.registers['force_sync'].write_int(1)